# Galleria rapida delle tracce

Notebook derivato da `View_tracking_tiles` per effettuare una revisione visiva del dataset di tracking.
Consente di sfogliare più clip contemporaneamente e di annotare velocemente quelle da conservare.

In [ ]:
import os, re
import numpy as np
import pandas as pd

from arguments import prepare_finetuning_args
from dataset.data_manager import BuildTrackingDataset, DataManager
import torch
from torch.utils.data import DataLoader
from dataset.datasets import MedicanesTrackDataset
from dataset.build_dataset import get_cyclone_center_pixel
from view_test_tiles import display_video_clip
from PIL import Image, ImageDraw

import ipywidgets as widgets
from IPython.display import display
from typing import Sequence, List, Optional, Any


In [ ]:
args = prepare_finetuning_args()

# Percorsi
input_dir = "../fromgcloud"
output_dir = "../airmassRGB/supervised/"
csv_out = "train_tracking.csv"          # CSV finale con pixel per tracking
manos_file = "medicane_data_input/more_medicanes_time_updated.csv"
df_tracks = pd.read_csv(manos_file, parse_dates=['time', 'start_time', 'end_time'])

bt = BuildTrackingDataset(type="supervised", args=args)
bt.prepare_data(df_tracks, input_dir, output_dir)
bt.create_tracking_csv(output_dir, csv_out)


In [ ]:
# Costruisce DataLoader per tracking a partire dal CSV generato
dm = DataManager(is_train=False, args=args, type_t='supervised', specify_data_path=csv_out)
track_loader = dm.get_tracking_dataloader(args)
track_ds = dm.dataset


## Selezione simultanea di più clip

La classe seguente crea una galleria di clip video con pulsanti "toggle" per selezionare le tracce da mantenere.
Gli identificativi selezionati (indici o path) possono poi essere salvati e riutilizzati per filtrare il DataFrame del CSV.

In [ ]:
class VideoGallerySelector:
    """Galleria interattiva per selezionare clip di tracking."""

    def __init__(
        self,
        clips: Sequence[Sequence[np.ndarray]],
        identifiers: Optional[Sequence[Any]] = None,
        columns: int = 3,
        interval: int = 200,
        toggle_icon: str = 'check'
    ) -> None:
        if identifiers is None:
            identifiers = list(range(len(clips)))
        if len(identifiers) != len(clips):
            raise ValueError('`identifiers` deve avere la stessa lunghezza di `clips`.')

        self.clips = clips
        self.identifiers = list(identifiers)
        self.columns = max(1, int(columns))
        self.interval = interval
        self.toggle_icon = toggle_icon

        self._toggles: List[widgets.ToggleButton] = []
        self._container = self._build_widget()

    def _build_widget(self) -> widgets.Widget:
        cards: List[widgets.Widget] = []
        grid_columns = min(self.columns, len(self.clips)) or 1
        column_template = f'repeat({grid_columns}, minmax(0, 1fr))'

        for idx, (clip, ident) in enumerate(zip(self.clips, self.identifiers)):
            output = widgets.Output(layout=widgets.Layout(width='100%'))
            with output:
                display(display_video_clip(clip, interval=self.interval))

            toggle = widgets.ToggleButton(
                value=False,
                description=str(ident),
                icon=self.toggle_icon,
                layout=widgets.Layout(width='100%')
            )
            toggle.observe(self._on_toggle_change, names='value')
            self._toggles.append(toggle)

            cards.append(
                widgets.VBox(
                    [output, toggle],
                    layout=widgets.Layout(
                        border='1px solid #ccc',
                        padding='6px',
                        align_items='stretch'
                    )
                )
            )

        grid = widgets.GridBox(
            cards,
            layout=widgets.Layout(
                grid_template_columns=column_template,
                grid_gap='12px',
                width='100%'
            )
        )

        self._selection_label = widgets.HTML()
        self._update_selection_label()

        return widgets.VBox([
            grid,
            self._selection_label
        ], layout=widgets.Layout(width='100%'))

    def _on_toggle_change(self, change):
        if change.get('name') == 'value':
            self._update_selection_label()

    def _update_selection_label(self) -> None:
        chosen = self.get_selected_identifiers()
        if chosen:
            values = ', '.join(map(str, chosen))
            self._selection_label.value = f"<b>Selezionati:</b> {values}"
        else:
            self._selection_label.value = '<i>Nessun video selezionato</i>'

    @property
    def widget(self) -> widgets.Widget:
        """Restituisce il contenitore principale da visualizzare nel notebook."""
        return self._container

    def display(self) -> None:
        """Visualizza la galleria nel notebook."""
        display(self._container)

    def get_selected_indices(self) -> List[int]:
        """Ritorna la lista degli indici (relativi alla galleria) selezionati."""
        return [idx for idx, toggle in enumerate(self._toggles) if toggle.value]

    def get_selected_identifiers(self) -> List[Any]:
        """Ritorna la lista degli identificativi associati alle clip selezionate."""
        indices = self.get_selected_indices()
        return [self.identifiers[idx] for idx in indices]

In [ ]:
# Estrae alcune clip dal dataset per popolare la galleria
max_samples = 6
sample_indices = list(range(max_samples))

clips = []
identifiers = []

mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1, 1)
std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1, 1)

for ds_idx in sample_indices:
    video, coords, folder = track_ds[ds_idx]
    video = (video * std + mean).clamp(0, 1)

    frames = []
    for frame in video.permute(1, 2, 3, 0).cpu().numpy():
        frames.append(np.clip(frame, 0.0, 1.0))
    clips.append(frames)

    # Usa il path della cartella come identificativo per poterlo incrociare con il CSV
    identifiers.append(folder)

selector = VideoGallerySelector(clips, identifiers=identifiers, columns=3, interval=200)
selector.display()


In [ ]:
# Recupera gli identificativi (es. path cartella) delle clip marcate
selezionati = selector.get_selected_identifiers()
selezionati


## Uso con il DataFrame del CSV

Gli identificativi ottenuti possono essere utilizzati per filtrare il CSV originale.
Ad esempio, se il path della cartella è presente in una colonna `folder`, si può eseguire:

```python
df_tracking = pd.read_csv(csv_out)
df_selezionati = df_tracking[df_tracking['folder'].isin(selezionati)]
```

In alternativa è possibile memorizzare gli indici relativi della galleria tramite
`selector.get_selected_indices()` e unirli con l'indice del DataFrame.